# AutoEncoder

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

In [2]:
(x_train, _), (x_test, _) = mnist.load_data()
x_train = x_train.reshape((60000,28,28,1))
x_train = x_train/255.0

11493376/11490434 [==============================] - 0s 0us/step


# Define Encoder.

In [3]:
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model

encoder_input = Input(shape=(28,28,1,), name='encoder_input')
x = encoder_input

encoder_conv_filters = [32,64,64,64]
encoder_conv_kernel_size = 3
encoder_conv_strides = [1,2,2,1]

for i in range(4):
  conv_layer = Conv2D(filters=encoder_conv_filters[i],
                      kernel_size = encoder_conv_kernel_size,
                      strides = encoder_conv_strides[i],
                      padding = 'same',
                      name = 'encoder_conv_' + str(i))
  
  x = conv_layer(x)
  x = LeakyReLU()(x)

shape_before_flattening = keras.backend.int_shape(x)[1:]
x = Flatten()(x)

encoder_output = Dense(2, name='encoder_output')(x)
encoder = Model(encoder_input, encoder_output, name='Encoder')
encoder.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "Encoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoder_input (InputLayer)   [(None, 28, 28, 1)]       0         
_________________________________________________________________
encoder_conv_0 (Conv2D)      (None, 28, 28, 32)        320       
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 28, 28, 32)        0         
_________________________________________________________________
encoder_conv_1 (Conv2D)      (None, 14, 14, 64)        18496     
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 14, 14, 64)        0         
_________________________________________________________________
encoder_conv_2 (Conv2D)      (None, 7, 7, 64)          36928     
____________________________________________

# Define Decoder

In [4]:
decoder_input = Input(shape=(2), name='decoder_input')

x = Dense(np.prod(shape_before_flattening))(decoder_input)
x = Reshape(shape_before_flattening)(x)

decoder_conv_t_filters = [64,64,32,1]
decoder_conv_t_kernel_size = 3
decoder_conv_t_strides = [1,2,2,1]

for i in range(4):
  conv_t_layer = Conv2DTranspose(
                      filters=decoder_conv_t_filters[i],
                      kernel_size = decoder_conv_t_kernel_size,
                      strides = decoder_conv_t_strides[i],
                      padding = 'same',
                      name = 'decoder_conv_t_' + str(i))
  
  x = conv_t_layer(x)
  if i < 4-1:
    x = LeakyReLU()(x)
  else:
    x = Activation('sigmoid')(x)

decoder_output = x

decoder = Model(decoder_input, decoder_output, name='Decoder')
decoder.summary()

Model: "Decoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
decoder_input (InputLayer)   [(None, 2)]               0         
_________________________________________________________________
dense (Dense)                (None, 3136)              9408      
_________________________________________________________________
reshape (Reshape)            (None, 7, 7, 64)          0         
_________________________________________________________________
decoder_conv_t_0 (Conv2DTran (None, 7, 7, 64)          36928     
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 7, 7, 64)          0         
_________________________________________________________________
decoder_conv_t_1 (Conv2DTran (None, 14, 14, 64)        36928     
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 14, 14, 64)        0   

# Autoencoder

In [5]:
from tensorflow.keras.utils import plot_model

model_input = encoder_input
model_output = decoder(encoder_output)
model = Model(model_input, model_output, name='AutoEncoder')

#plot_model(model, show_shapes=True)
model.summary()

Model: "AutoEncoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoder_input (InputLayer)   [(None, 28, 28, 1)]       0         
_________________________________________________________________
encoder_conv_0 (Conv2D)      (None, 28, 28, 32)        320       
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 28, 28, 32)        0         
_________________________________________________________________
encoder_conv_1 (Conv2D)      (None, 14, 14, 64)        18496     
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 14, 14, 64)        0         
_________________________________________________________________
encoder_conv_2 (Conv2D)      (None, 7, 7, 64)          36928     
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 7, 7, 64)          

# Compile
Compile using Adam optimizer with 0.0005 learning rate, Root Mean Squared Error:RMSE.

In [0]:
from tensorflow.keras.optimizers import Adam

optimizer = Adam(lr=0.005)

def r_loss(y_true, y_pred):
  return keras.backend.mean(keras.backend.square(y_true - y_pred), axis = [1,2,3])

model.compile(optimizer=optimizer, loss=r_loss)

In [7]:
model.fit(
    x = x_train,
    y = x_train,
    batch_size = 32,
    shuffle = True,
    epochs = 10,
    verbose = 2
)

Train on 60000 samples
Epoch 1/10
60000/60000 - 28s - loss: 0.1127
Epoch 2/10
60000/60000 - 21s - loss: 0.1120
Epoch 3/10
60000/60000 - 21s - loss: 0.1120
Epoch 4/10


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-7-9867f04efcd0>", line 7, in <module>
    verbose = 2
  File "/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/keras/engine/training.py", line 727, in fit
    use_multiprocessing=use_multiprocessing)
  File "/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/keras/engine/training_arrays.py", line 675, in fit
    steps_name='steps_per_epoch')
  File "/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/keras/engine/training_arrays.py", line 394, in model_iteration
    batch_outs = f(ins_batch)
  File "/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/keras/backend.py", line 3476, in __call__
    run_metadata=self.run_metadata)
  File "/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/client/session.py", line 1472, in 

KeyboardInterrupt: ignored